# Differential expression in R

In [ ]:
R.version
R.home()

We will use the `tidyverse` family of packages and `DESeq2` to do this analysis. In R, we load packages like this:

In [ ]:
library(tidyverse)
library(DESeq2)

## QC on raw counts

Before we do any differential expression analysis, it's good to do some QC on the count data to ensure the sequencing samples cluster as expected. For our experiment, we expect four clusters (LIV_ma, LIV_im, INT_ma, and INT_im) each with three points.

### Gene filtering based on total read count and the number of samples the gene is found in

Before we do anything, we are going to filter the count data to remove genes with low total expression values (anything less than 10 is not going to be informative) and genes that were only expressed in 3 or fewer samples. First, we read in the counts data and convert numeric columns to integers:

In [ ]:
counts_df <- read_tsv("/data/classes/2024/fall/biol343/course_files/star_counts.tsv",
                      comment = "#") |>
             mutate(across(where(is.numeric), as.integer))
counts_df

Next, we do the filtering. The first operation will sum the expression across all samples and remove any gene that had a total count of less than or equal to 10. The second operation will count each sample that had counts >0 of each gene. Any gene that wasn't expressed in >3 samples will be removed entirely.

In [ ]:
counts_summary <- counts_df |>
    select(Geneid, contains('star.bam')) |>
    rename_with(~str_remove(., "/data/classes/2024/fall/biol343/course_files/dedup/star.bam:"), everything()) |>
    rowwise() |>
    mutate(total_counts = sum(c_across(where(is.numeric)), na.rm = T)) |>
    filter(total_counts >= 10)
counts_summary

sample_summary <- counts_df |>
    select(Geneid, contains('star.bam')) |>
    rename_with(~str_remove(., "/data/classes/2024/fall/biol343/course_files/dedup/star.bam:"), everything()) |>
    pivot_longer(-Geneid, names_to = 'sample', values_to = 'count') |>
    filter(count > 0) |>
    group_by(Geneid) |>
    tally() |>
    filter(n <= 3)
sample_summary

genes_to_remove = sample_summary$Geneid

counts_filt <- counts_summary |>
    filter(!Geneid %in% genes_to_remove) |>
    arrange(Geneid) |>
    select(-total_counts)
counts_filt

You can see that in the initial data (`count_m`), there were 9920 rows (genes). In `counts_summary`, there are 9000. Which means that 920 genes were removed because they were lowly expressed. There are 8991 rows in `counts_filt`, which means an additional 9 genes were removed because they showed up in <3 samples.

This filtered dataset will be used for all downstream analyses. First, we'll measure the Euclidean distance between each sample and then plot the results as a heatmap. We'll first convert the data from a data frame to a matrix, and then measure the distance:

In [ ]:
counts_m <- counts_filt |>
    select(-Geneid) |>
    as.matrix()
rownames(counts_m) <- counts_filt$Geneid
counts_m

dists <- dist(t(counts_m))
dists

We can then use these distances to plot the heatmap:

In [ ]:
dists_df <- as.matrix(dists) |>
    as_tibble(rownames = 'sample')

dist_plot <- dists_df |>
    pivot_longer(-sample, names_to = 'comp', values_to = 'dist') |>
    ggplot(aes(x = sample, y = comp, fill = dist)) +
    geom_tile() +
    scale_fill_viridis_c() +
    coord_equal() +
    NULL
dist_plot

We can clearly see 3 main clusters - LIV_ma, LIV_im, and INT. The difference between INT_ma and INT_im is more difficult to see. Let's see how a PCA looks:

In [ ]:
pca_fit <- t(log10(counts_m + 1)) |> 
  prcomp(scale = TRUE)
pca_fit

Now plot the first two PCs:

In [ ]:
library(broom)

pca_fit |>
  augment(t(counts_m)) |>
  dplyr::rename(sample = .rownames) |>
  separate(sample, into = c('tissue', 'age'), sep = '_') |>
  mutate(age = str_remove(age, '[0-9]')) |>
  ggplot(aes(.fittedPC1, .fittedPC2, color = tissue, shape = age)) + 
  geom_point(size = 4)

Again, we're seeing one strange replicate in the INT_ma data. In the publication, they say in their methods:

>For PCA within DESeq2 package, log–transformed read count data was used as an input.

So, they didn't perform PCA on the read counts, but on transformed data from the DESeq2 analysis. We will run PCA again after using DESeq2 to see how it differs.



## Differential expression analysis
To perform differential expression analysis, DESeq2 requires two types of input:

1. A count matrix of shape ‘number of samples’ x ‘number of genes’, containing read counts (non-negative integers) 
2. Metadata (or “column” data) of shape ‘number of samples’ x ‘number of variables’, containing sample annotations that will be used to split the data in cohorts.

The output of featureCounts needs to be converted to a matrix, the column names should be simplified, and a few unnecessary columns need to be removed. We did most of that earlier (creating `counts_m`). Below, we'll create the metadata data frame:

In [ ]:
metadata <- data.frame(sample_id = colnames(counts_m)) |>
    mutate(tissue = str_sub(sample_id, 1, 3),
           age = str_sub(sample_id, 5, 6),
           rep = str_sub(sample_id, 7))
rownames(metadata) <- metadata$sample_id
metadata <- select(metadata, -sample_id)
metadata

Check to confirm the metadata and count columns from `counts_m` correspond.

In [ ]:
all(rownames(metadata) == colnames(counts_m))

### Single factor analyses

DESeq2 allows for single factorial analyses (only one independent variable, i.e., just tissue) or multifactorial analyses (more than independent variable, i.e., tissue and age). We will first analyze the data just to see if there are differences between liver and intestine eggs, ignoring the age of the eggs. To do so, we create a `DeseqDataSet` (or DDS object), which incorporates the counts and the metadata. We can then run the `deseq()` method on the DDS object to fit dispersions and log-fold changes:

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = counts_m,
                              colData = metadata,
                              design = ~ tissue)
dds <- DESeq(dds)
dds

We can run the `results()` method on the `dds` objects, which runs the statistical tests. Now, for example, we can access the gene-level LFCs (stored in `res` below) and look at the results for genes of interest:

In [ ]:
res <- results(dds)
res

as_tibble(res, rownames = 'gene_id') |>
    filter(gene_id == 'Smp_329140')

For the gene Smp_329140, we calculated that there is a 2^1.147953 = 2.216 fold change between LIV and INT eggs (higher in liver). Below are the expression values that were counted with featureCounts. Does a ~2-fold difference look right?

In [ ]:
filter(counts_filt, Geneid == 'Smp_329140')

The expression values for LIV were 2, 0, 3, 1, 0, and 3 (mean of 1.5). The expression values for INT were 0, 0, 0, 0, 0, 1 (mean of 0.167). 1.5 / 0.167 is right around 9. So, our L2FC calculation is slightly lower than the raw L2FC of 9, which makes sense based on what we know about how DESeq2 handles expression values that are low counts (it reduces them).

One of the featured differentially expressed genes from the Winners vs. Losers paper was Smp_333930, which encodes for an immunomodulatory molecule IPSE/alpha-1. We can search for that gene to see if there was a difference between INT and LIV expression when not accounting for age:

In [ ]:
as_tibble(res, rownames = 'gene_id') |>
    filter(gene_id == 'Smp_333930')

This gene is significantly differentially expressed when taking all LIV and INT eggs together. Let's now look at the results when we separate Tissue and Age, giving us four total groups to compare.

### Multifactorial analysis

To do a multifactorial analysis, we have to remake the DDS object, this time changing the `design` parameter to include both tissue and age. We'll update the metadata and join the tissue and age columns, which will now have the original 4 factors (LIV_im, LIV_ma, INT_im, and INT_ma).

In [ ]:
metadata2 <- metadata |>
    mutate(tissue_age = str_c(tissue, "_", age)) |>
    select(tissue_age, rep)
metadata2

dds2 <- DESeqDataSetFromMatrix(countData = counts_m,
                               colData = metadata2,
                               design = ~ tissue_age)
dds2 <- DESeq(dds2)
dds2

When there are >2 levels in the variable (like ours, which has four) DESeq2 will automatically compare the first and last levels unless we provide the comparison we want with the `contrast` argument of the `results()` method. Below, we will calculate differnces betweeen LIV_ma and INT_ma, which was one of the main comparisons in the paper (Fig. 4):

In [ ]:
LIV_ma_vs_INT_ma <- results(dds2, contrast = c('tissue_age', 'LIV_ma', 'INT_ma'))
LIV_ma_vs_INT_ma

And now we can again check our gene of interest:

In [ ]:
as_tibble(LIV_ma_vs_INT_ma, rownames = 'gene_id') |>
    filter(gene_id == 'Smp_333930')

The gene is expressed much higher in LIV_ma eggs than INT_ma eggs (5.59 log fold change, or 2^5.59 = ~50 fold difference), but the difference isn't quite significant (padj of 0.056 > 0.05).

### Volcano plots

Typically in RNA-seq analyses, we are interested in genes that are significantly differentially expressed (padj <0.05) and those that have large differences (maybe log two fold change > 2, which is a four-fold change). Volcano plots are a good way to visualize all the genes that satisfy both conditions.

In [ ]:
volcano_data <- as_tibble(LIV_ma_vs_INT_ma, rownames = 'gene_id')

volcano_plot <- volcano_data |> 
    ggplot(aes(x = log2FoldChange, y = -log10(padj))) +
    geom_point() +
    geom_hline(yintercept = -log10(0.05)) +
    geom_vline(xintercept = 2) +
    geom_vline(xintercept = -2) +
    theme_minimal()
volcano_plot

In a plot like this, the higher points represent genes that are most likely to be truly significantly differentially expressed (a small p-value). The points far to the right or left represent genes that have large expression differences (fold changes) between LIV_ma and INT_ma. Genes on the right are more highly expressed in LIV_ma (positive log2FoldChange), and the genes on the left are more highly expressed in INT_ma eggs (negative log2FoldChange). The points in the top right/left areas, then, are the genes in which we're interested.

## QC on transformed data

Now what we have the `dds` object, we can do another PCA to see if we can get something that looks like their Fig. 1D. We first transform the read counts with DESeq2's variance-stabilizing transformation:

In [ ]:
vsd <- vst(dds)
vsd

DESeq2 includes a method to easily plot a PCA:

In [ ]:
plotPCA(vsd, intgroup = c("tissue", "age"))

We can see that this fixed the weird result we saw earlier on raw read counts. The variance-stabilizing transformation (`vst`) provided by DESeq2 results in clustering that makes sense. Now we see tissue represented by PC2 and age by PC1.

We can check the distance results too:

In [ ]:
vsd_dists <- dist(t(assay(vsd)))

vsd_dists_df <- as.matrix(vsd_dists) |>
    as_tibble(rownames = 'sample')

vsd_dist_plot <- vsd_dists_df |>
    pivot_longer(-sample, names_to = 'comp', values_to = 'dist') |>
    ggplot(aes(x = sample, y = comp, fill = dist)) +
    geom_tile() +
    scale_fill_viridis_c() +
    coord_equal() +
    NULL
vsd_dist_plot